In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
artist = pd.read_csv(r'data/artist.csv')
canvas_Size = pd.read_csv(r'data/canvas_size.csv')
image_link = pd.read_csv(r'data/image_link.csv')
museum_hours = pd.read_csv(r'data/museum_hours.csv')
museum = pd.read_csv(r'data/museum.csv')
product_size = pd.read_csv(r'data/product_size.csv')
subject = pd.read_csv(r'data/subject.csv')
work = pd.read_csv(r'data/work.csv')

In [3]:
# 1. Fetch all the paintings which are not displayed in any museums
paintings_not_in_museums = work[work['museum_id'].isna()]
paintings_not_in_museums

,work_id,name,artist_id,style,museum_id
6,125752,Arabian Horses at Pasture,757,Baroque,NaN
7,125818,Count Halm on His Basedow Estate,757,Baroque,NaN
9,125763,Napoleon Before the Burning City of Smolensk,757,Baroque,NaN
10,125774,Peasants Resting in the Field,757,Baroque,NaN
11,125785,Portrait Oberleutnant Theodor Von Klein,757,Baroque,NaN
...,...,...,...,...,...
14768,8273,Waiting for an Answer,794,American Art,NaN
14772,8283,"West Point, Prout's Neck",794,American Art,NaN
14773,8285,Winding Line,794,American Art,NaN
14774,184820,Untitled,620,Surrealism,NaN


In [4]:
# 2. Are there museums without any paintings?
museums_with_paintings = work['museum_id'].dropna().unique()
museums_without_paintings = museum[~museum['museum_id'].isin(museums_with_paintings)]
museums_without_paintings

,museum_id,name,address,city,state,postal,country,phone,url


In [5]:
# 3. How many paintings have an asking price of more than their regular price?
paintings_higher_asking_price = product_size[product_size['sale_price'] > product_size['regular_price']]
paintings_higher_asking_price

,work_id,size_id,sale_price,regular_price


In [6]:
# 4. Identify the paintings whose asking price is less than 50% of its regular price
paintings_asking_less_than_50_percent = product_size[product_size['sale_price'] < 0.5 * product_size['regular_price']]
paintings_asking_less_than_50_percent

,work_id,size_id,sale_price,regular_price
220,31780,36,10,125
221,31780,30,10,95
686,31780,36,10,125
687,31780,30,10,95
14073,198417,36,30,125
14074,198417,30,30,95
17331,31974,24,30,85
29406,17351,24,10,85
29407,17351,30,10,95
29408,17351,36,10,125


In [7]:
# 5. Identify the canvas size that costs the most
max_cost_size_id = product_size.groupby('size_id')['sale_price'].max().idxmax()
most_expensive_canvas_size = canvas_Size[canvas_Size['size_id'] == int(max_cost_size_id)]
print(most_expensive_canvas_size)

     size_id  width  height                       label
161     4896     48    96.0  48" x 96"(122 cm x 244 cm)


In [8]:
# 6. Delete duplicate records from work, product_size, subject, and image_link tables
work.drop_duplicates(inplace=True)
product_size.drop_duplicates(inplace=True)
subject.drop_duplicates(inplace=True)
image_link.drop_duplicates(inplace=True)

# Save cleaned files
work.to_csv('cleaned_work.csv', index=False)
product_size.to_csv('cleaned_product_size.csv', index=False)
subject.to_csv('cleaned_subject.csv', index=False)
image_link.to_csv('cleaned_image_link.csv', index=False)

In [9]:
# 7. Identify museums with invalid city information
invalid_cities = museum[museum['city'].isna() | (museum['city'].str.strip() == '')]
print(invalid_cities)

Empty DataFrame
Columns: [museum_id, name, address, city, state, postal, country, phone, url]
Index: []


In [10]:
# 8. Identify and remove the invalid entry in the museum_hours table
invalid_hours = museum_hours[(museum_hours['open'].isna()) | (museum_hours['close'].isna())]
museum_hours_cleaned = museum_hours.drop(invalid_hours.index)
print(invalid_hours)
museum_hours_cleaned.to_csv('cleaned_museum_hours.csv', index=False)

Empty DataFrame
Columns: [museum_id, day, open, close]
Index: []


In [11]:
# 9. Fetch the top 10 most famous painting subjects
top_subjects = subject['subject'].value_counts().head(10)
print(top_subjects)

Portraits              1070
Nude                    525
Landscape Art           495
Rivers/Lakes            480
Flowers                 457
Abstract/Modern Art     399
Still-Life              395
Seascapes               323
Marine Art/Maritime     268
Horses                  265
Name: subject, dtype: int64


In [12]:
# 10. Identify museums open on both Sunday and Monday and display their names and cities
sunday_open = museum_hours[(museum_hours['day'] == 'Sunday') & (museum_hours['open'].notna())]
monday_open = museum_hours[(museum_hours['day'] == 'Monday') & (museum_hours['open'].notna())]
open_both_days = pd.merge(sunday_open, monday_open, on='museum_id')
museums_open_both_days = museum[museum['museum_id'].isin(open_both_days['museum_id'])]
print(museums_open_both_days[['name', 'city']])

                                 name          city
0            The Museum of Modern Art      New York
1   Pushkin State Museum of Fine Arts        Moscow
2        National Gallery of Victoria     Melbourne
5      The Metropolitan Museum of Art      New York
7                  Museum of Grenoble         38000
9         Nelson-Atkins Museum of Art   Kansas City
10                    Musée du Louvre         75001
11           National Maritime Museum        London
12         Museum of Fine Arts Boston        Boston
13                        Rijksmuseum     Amsterdam
14                      Israel Museum     Jerusalem
16            National Gallery of Art    Washington
17                   National Gallery        London
18                 Mauritshuis Museum      Den Haag
20                   The Prado Museum        Madrid
21              The Barnes Foundation  Philadelphia
27                    Van Gogh Museum     Amsterdam
30   Los Angeles County Museum of Art   Los Angeles
31       Sol

In [13]:
# 11. Count how many museums are open every single day
days_open = museum_hours.groupby('museum_id')['day'].nunique()
museums_open_every_day = days_open[days_open == 7].index
count_museums_open_every_day = len(museums_open_every_day)
print(count_museums_open_every_day)

17


In [14]:
# 12. Top 5 most popular museums (based on the number of paintings)
museum_paintings_count = work['museum_id'].value_counts().head(5)
top_5_museums = museum[museum['museum_id'].isin(museum_paintings_count.index)]
print(top_5_museums[['name', 'city']])

                              name          city
5   The Metropolitan Museum of Art      New York
13                     Rijksmuseum     Amsterdam
16         National Gallery of Art    Washington
17                National Gallery        London
21           The Barnes Foundation  Philadelphia


In [15]:
# 13. Top 5 most popular artists (based on the number of paintings)
artist_paintings_count = work['artist_id'].value_counts().head(5)
top_5_artists = artist[artist['artist_id'].isin(artist_paintings_count.index)]
print(top_5_artists[['full_name']])

                 full_name
0    Pierre-Auguste Renoir
50            Claude Monet
59          Albert Marquet
149        Maurice Utrillo
177       Vincent Van Gogh


In [16]:
# 14. Display the 3 least popular canvas sizes
canvas_paintings_count = product_size['size_id'].value_counts().tail(3)
print(canvas_paintings_count)

5062.6    1
4235.5    1
35.628    1
Name: size_id, dtype: int64


In [17]:
# 16. Museum with the most number of the most popular painting style
most_popular_style = work['style'].value_counts().idxmax()
museums_most_popular_style = work[work['style'] == most_popular_style]['museum_id'].value_counts()
top_museum_for_popular_style = museum[museum['museum_id'] == museums_most_popular_style.idxmax()]
print(top_museum_for_popular_style)


   museum_id                            name       address      city state  \
5         35  The Metropolitan Museum of Art  1000 5th Ave  New York    NY   

  postal country            phone                         url  
5  10028     USA  +1 212 535-7710  https://www.metmuseum.org/  


In [18]:
# 17. Artists with paintings in multiple countries
artist_countries = work.merge(museum, on='museum_id').groupby('artist_id')['country'].nunique()
artists_multiple_countries = artist_countries[artist_countries > 1].index
artists_with_multiple_countries = artist[artist['artist_id'].isin(artists_multiple_countries)]
print(artists_with_multiple_countries[['full_name']])


                       full_name
0          Pierre-Auguste Renoir
1              Alexandre Cabanel
3                Maximilien Luce
4                   August Macke
5            Thomas Gainsborough
..                           ...
412               Pierre Bonnard
413           Edward Burne-Jones
416  Jean-Baptiste-Camille Corot
419       Edmund Charles Tarbell
420           Rembrandt Van Rijn

[194 rows x 1 columns]


In [19]:
# 18. Country and city with the most number of museums
most_museums_country = museum['country'].value_counts().idxmax()
most_museums_city = museum['city'].value_counts().idxmax()
print(f"Country with most museums: {most_museums_country}")
print(f"City with most museums: {most_museums_city}")

Country with most museums: USA
City with most museums: New York


In [20]:
# 19. Artist and museum with the most expensive and least expensive painting
most_expensive_painting = product_size.loc[product_size['sale_price'].idxmax()]
least_expensive_painting = product_size.loc[product_size['sale_price'].idxmin()]

def get_painting_details(painting):
    painting_details = work[work['work_id'] == painting['work_id']].iloc[0]
    artist_details = artist[artist['artist_id'] == painting_details['artist_id']].iloc[0]
    
    # Check if museum_id is not NaN and exists in the museum DataFrame
    if pd.notna(painting_details['museum_id']) and not museum[museum['museum_id'] == painting_details['museum_id']].empty:
        museum_details = museum[museum['museum_id'] == painting_details['museum_id']].iloc[0]
    else:
        museum_details = {'name': 'N/A', 'city': 'N/A'}
    
    # Check if size_id exists in the canvas_Size DataFrame
    if not canvas_Size[canvas_Size['size_id'] == painting['size_id']].empty:
        canvas_details = canvas_Size[canvas_Size['size_id'] == painting['size_id']].iloc[0]
    else:
        canvas_details = {'label': 'N/A'}
    
    return {
        'artist_name': artist_details['full_name'],
        'sale_price': painting['sale_price'],
        'painting_name': painting_details['name'],
        'museum_name': museum_details['name'],
        'museum_city': museum_details['city'],
        'canvas_label': canvas_details['label']
    }

most_expensive_painting_details = get_painting_details(most_expensive_painting)
least_expensive_painting_details = get_painting_details(least_expensive_painting)

print(most_expensive_painting_details)
print(least_expensive_painting_details)


{'artist_name': 'Peter Paul Rubens', 'sale_price': 1115, 'painting_name': 'Fortuna', 'museum_name': 'The Prado Museum', 'museum_city': 'Madrid', 'canvas_label': 'N/A'}
{'artist_name': 'Adélaïde Labille-Guiard', 'sale_price': 10, 'painting_name': 'Portrait of Madame Labille-Guyard and Her Pupils', 'museum_name': 'The Metropolitan Museum of Art', 'museum_city': 'New York', 'canvas_label': 'N/A'}


In [21]:
# 20. Country with the 5th highest number of paintings
country_paintings_count = work.merge(museum, on='museum_id').groupby('country')['work_id'].count()
fifth_highest_country = country_paintings_count.sort_values(ascending=False).index[4]
print(f"Country with the 5th highest number of paintings: {fifth_highest_country}")

Country with the 5th highest number of paintings: Spain


In [22]:
# 21. 3 most popular and 3 least popular painting styles
painting_styles_count = work['style'].value_counts()
most_popular_styles = painting_styles_count.head(3)
least_popular_styles = painting_styles_count.tail(3)
print("3 Most popular styles:", most_popular_styles)
print("3 Least popular styles:", least_popular_styles)

3 Most popular styles: Impressionism         3078
Post-Impressionism    1672
Realism               1179
Name: style, dtype: int64
3 Least popular styles: Avant-Garde     146
Art Nouveau     108
Japanese Art     70
Name: style, dtype: int64


In [23]:
# 22. Artist with the most Portrait paintings outside the USA
# Filter for Portrait paintings outside the USA
portraits_outside_usa = work[work['style'] == 'Portrait'].merge(museum, on='museum_id')
portraits_outside_usa = portraits_outside_usa[portraits_outside_usa['country'] != 'USA']

# Check if there are any Portrait paintings outside the USA
if not portraits_outside_usa.empty:
    # Get the artist with the most Portrait paintings outside the USA
    artist_portraits_outside_usa = portraits_outside_usa['artist_id'].value_counts().idxmax()
    artist_details = artist[artist['artist_id'] == artist_portraits_outside_usa].iloc[0]
    artist_portraits_count = portraits_outside_usa[portraits_outside_usa['artist_id'] == artist_portraits_outside_usa].shape[0]
    
    # Print the details
    print(f"Artist with most Portrait paintings outside USA: {artist_details['full_name']}, Count: {artist_portraits_count}, Nationality: {artist_details['nationality']}")
else:
    print("No Portrait paintings found outside the USA.")


No Portrait paintings found outside the USA.
